# Giới thiệu 
- Notebook này là phần được ghi chú lại trong quá học khóa học [rag-from-scratch](https://github.com/langchain-ai/rag-from-scratch?tab=readme-ov-file) về việc xây dựng mô hình RAG từ đầu
- Trong phần này chủ yếu giới thiệu về cách xây dựng một pipeline cơ bản nhất cho một hệ thống RAG, pipeline bao gồm 3 phần chính như sau: 
<p align="center">
    <img src="../doc/image/basic-image.png" alt="basic-pipeline" width="400"/>
</p>




- Trong đó : 
1. Indexing: Phần dữ liệu nền tảng sẽ được thu thập và tách, sau đó được indexing vào trong một vector database
2. Retrieval: phần này xử lý dữ liệu người dùng (user query) sẽ được index vào trong vector database để tìm kiếm được k chunks liên quan nhất tới query 
3. Generation : phần này chuyển đổi yêu cầu của người dùng và lượng dữ liệu của đã được trích xuất tù phần retrieval để tạo ra response với trả lời câu hỏi của người dùng dựa trên kiến thức được trích xuất.   

# Triển khai Basic pipeline
- Triển khai một pipeline của RAG dựa trên : 
    1. Mô hình LLM và Embedding : Gemini
    2. Dữ liệu indexing : [wiki - Faker](https://en.wikipedia.org/wiki/Faker_(gamer)) 
    3. Trích xuất để trả lời câu hỏi "How many World Championship titles has Faker won in his League of Legends career?" 

## Setup 

In [ ]:
# import thư viện cần thiết 
import bs4
from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_google_genai import GoogleGenerativeAIEmbeddings, ChatGoogleGenerativeAI

In [ ]:
import os 
from dotenv import load_dotenv

load_dotenv()

GOOGLE_API_KEY = os.getenv("GEMINI_API_KEY")
LANGCHAIN_TRACING_V2 = os.getenv("LANGCHAIN_TRACING_V2")
LANGCHAIN_ENDPOINT = os.getenv("LANGCHAIN_ENDPOINT")
LANGCHAIN_API_KEY = os.getenv("LANGCHAIN_API_KEY")

## Indexing 


In [ ]:
# Load Documents từ trang wikipedia
loader = WebBaseLoader(
    web_paths=("https://en.wikipedia.org/wiki/Faker_(gamer)",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("mw-body-content")
        )
    ),
)
docs = loader.load()

# Split chia đoạn document được load
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

# Embedding đoạn dữ liệu vào trong vector database thông qua mô hình Gemini
vectorstore = Chroma.from_documents(documents=splits,
                                    embedding=GoogleGenerativeAIEmbeddings(model="models/embedding-001",  google_api_key = GOOGLE_API_KEY ))

retriever = vectorstore.as_retriever()

## Retrieval and Generation 

In [ ]:
# Prompt được lấy từ hub có sẵn
prompt = hub.pull("rlm/rag-prompt")

# Load LLm model
llm = ChatGoogleGenerativeAI(model = 'gemini-1.5-pro-latest',
                                       temperature=0.2,
                                       api_key= GOOGLE_API_KEY)

# Post-processing hậu xử lý văn bản
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Chain
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# Question
rag_chain.invoke("How many World Championship titles has Faker won in his League of Legends career?")